In [ ]:
import json
import boto3
import logging

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def lambda_handler(event, context):
    body = json.loads(event['body'])
    user_input = body.get("user_input", "")

    if not user_input:
        return {
            'statusCode': 400,
            'body': json.dumps({"error": "User input required"})
        }

    logger.info(f"User Input: {user_input}")

    try:
        client = boto3.client(service_name="bedrock-runtime", region_name="<your-region>")  # e.g., us-east-1
        model_id = "anthropic.claude-v2"

        request_body = json.dumps({
            "prompt": f"\n\nHuman: {user_input}\n\nAssistant:",
            "max_tokens_to_sample": 500,
            "temperature": 0.7,
            "top_k": 50,
            "top_p": 0.9
        })

        response = client.invoke_model(
            modelId=model_id,
            body=request_body
        )

        result = json.loads(response["body"].read().decode("utf-8"))
        ai_response = result.get("completion", "Sorry, I couldn't generate a response.")

        # Log AI response
        logger.info(f"AI Response: {ai_response}")

        return {
            'statusCode': 200,
            'headers': {'Content-Type': 'application/json'},
            'body': json.dumps({"response": ai_response})
        }

    except Exception as e:
        logger.error(f"Error: {str(e)}")
        return {
            'statusCode': 500,
            'body': json.dumps({"error": str(e)})
        }
